In [1]:
import matplotlib.pyplot as plt
import argparse
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import numpy as np
from models import *
import torch
import torch.optim
from skimage import io
from skimage.metrics import structural_similarity as compare_ssim
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
import cv2
from utils.sr_utils import *
from models.down_otf_nn import down_otf_nn
from utils.get import get_AiryDisk as get_otf
from utils.get import get_3d 
from utils.tv_loss import tv_loss
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

imsize = -1 
PLOT = True
filename=r'E:\DIP_shuju\515/' #Image folder location
save_path=filename+'7_1/'## Folder location for output
os.makedirs(save_path, exist_ok=True)
path_to_image1 = filename+'7.tif'## Low resolution image
path_to_image2=path_to_image1 
Multiple_pixel=1 ### Need to double the pixels?


KeyboardInterrupt: 

In [ ]:
if torch.cuda.is_available():
    dtype = torch.float
    device = 'cuda' 
    torch.cuda.empty_cache()
else:
    dtype = torch.float
    device = 'cpu'
print(device)

In [ ]:
imgs = load_img(path_to_image1,path_to_image2 , imsize=-1)
######
## Microscope system parameter
crop_img_size=imgs['HR_pil'].size[1]
noise_gaussian = torch.randn(10,1000,1000)  
noise_gaussian=noise_gaussian
noise_gaussian=noise_gaussian.unsqueeze(0)
noise_gaussian=noise_gaussian.type(dtype)
####
##########
input_depth=10
output_channels=1
net = get_net(input_depth, 'skip', 'reflection',
              output_channels=1,
              skip_n33d=64, 
              skip_n33u=64, 
              skip_n11=4, 
              num_scales=5,
              upsample_mode='bilinear').type(dtype)
##########

In [ ]:
tv_w=0  #0.001
temp_data=imgs['LR_np'][0,:,:]
mse = torch.nn.MSELoss().type(dtype)
mae=torch.nn.L1Loss().type(dtype)
temp_data2=temp_data[None,:,:]
img_LR_var = np_to_torch(temp_data2).type(dtype)
net_input=noise_gaussian 
net=net.to(device) 
net_input=net_input.to(device) 
img_LR_var=img_LR_var.to(device) 

In [ ]:
def Iteration():
    global i,net_input
    out_HR = net(net_input)
    total_loss =mse(out_HR, img_LR_var)
    TV_l=tv_w * tv_loss(out_HR)
    total_loss += TV_l
    total_loss.backward()
    # Log
    psnr_LR = compare_psnr(imgs['LR_np'], imgs['HR_np'])
    ssim_LR=compare_ssim(imgs['HR_np'][0,:,:],imgs['LR_np'][0,:,:],data_range=65535)
    psnr_value.append([psnr_LR])
    print(f"Epoch {i}, Loss: {total_loss.item()},TV_Loss: {TV_l.item()}")
    i += 1
    if PLOT and i % 100 == 0:
        out_HR_np = torch_to_np(out_HR)
        out_HR_np1=out_HR_np
        plot_image_grid([temp_data2, out_HR_np1], factor=13, nrow=2)

    if PLOT and i % 10 == 0:
        out_out = np.clip(torch_to_np(net(net_input)), 0, 1)
       # print(np.max(np.max(out_out)))
        out=out_out*65535/1
        #out=out_out
        out=out.astype('uint16')
        if i>=10 and i<100:
            io.imsave(save_path+'/sim_HR_iter'+'00'+str(i)+'.tif',out)
        elif i>=100 and i<1000:
            io.imsave(save_path+'/sim_HR_iter'+'0'+str(i)+'.tif',out)
        elif i>=1000:
            io.imsave(save_path+'/sim_HR_iter'+'0'+str(i)+'.tif',out)
    return total_loss
def get_params1(net):
    params = []
    params += [x for x in net.parameters()]
    return params
def opt(para_temp,Iteration, LR, num_iter):
    optimizer = torch.optim.Adam(para_temp, lr=LR)
    for j in range(num_iter):
        optimizer.zero_grad()
        Iteration()
        optimizer.step()

In [ ]:
psnr_value = [] 
net_input_saved = net_input.detach().clone()
noise = net_input.detach().clone()
num_iter = 2000
i = 0

LR=0.01
opt(get_params1(net),Iteration, LR, num_iter)